In [1]:
from graph_tool import *
from tqdm import tqdm
import csv

In [2]:
# get dict with REMID as key and corresponding CREM as value
# dict with CREM as key and contained REMs as value
crem_dict = dict()
rem_crem_dict = dict()
with open('../data/clusterREMs_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        if row[0] != "REMID":
            crem_dict[row[0]] = [row[1], row[3], row[4]]
            if rem_crem_dict.get(row[1]) == None:
                rem_crem_dict[row[1]] = [row[0]]
            else:
                rem_crem_dict[row[1]].append(row[0])

1264526it [00:05, 247728.50it/s]


In [3]:
# get dict with GENE name as key and start and end as value
gene_dict = dict()
with open('../data/GeneAnnotation_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        if row[0] != "chr":
            gene_dict[row[3]] = [int(row[1]), int(row[2]), row[0]]

58175it [00:00, 406372.69it/s]


In [4]:
sample_celltype_dict = dict()
ID_celltype_dict = dict()
with open('../data/sampleToCelltype.csv') as sampleToCelltype:
    reader = csv.reader(sampleToCelltype)
    for row in reader:
        if row[0] != "sampleID":
            # load dict with sample and corresponding celltype
            sample_celltype_dict[row[0]] = [row[1], row[2]]
            
            # load dict with celltype ID and corresponding celltype
            ID_celltype_dict[row[1]] = row[2]

# print(sample_celltype_dict)
# print(ID_celltype_dict)

In [5]:
# dict for genes with their expression values in each sample for both data sets (Roadmap and Blueprint)
gene_expr_dict = dict()
with open('../data/GeneExpressionRoadmap_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        species = row[3]
        gene = row[0]
        sample = row[1]
        exprs = row[2]
#         if sample not in sample_set:
#             continue
        if species != "species":
            if gene_expr_dict.get(gene) == None:
                gene_expr_dict[gene] = {sample : exprs}
            elif gene_expr_dict[gene].get(sample) == None:
                gene_expr_dict.get(gene)[sample] = exprs
                
with open('../data/GeneExpressionBlueprint_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        species = row[3]
        gene = row[0]
        sample = row[1]
        exprs = row[2]
#         if sample not in sample_set:
#             continue
        if species != "species":
            if gene_expr_dict.get(gene) == None:
                gene_expr_dict[gene] = {sample : exprs}
            elif gene_expr_dict[gene].get(sample) == None:
                gene_expr_dict.get(gene)[sample] = exprs


6399141it [00:09, 644180.88it/s]
3257745it [00:04, 667813.42it/s]


In [6]:
# rem_dict{celltypes : {samples: {REMs: score}}}
# dict to map celltype to samples and the included REMs with their scores

rem_dict = dict()
with open('../data/REMActivity_1.csv') as file_1:
    reader_1 = csv.reader(file_1)
    for r in tqdm(reader_1):
        rem = r[0] 
        sample = r[1] 
        score = r[3] 
        
        if sample_celltype_dict.get(sample) == None:       # if sample not 
            continue
        else:
            celltype = sample_celltype_dict.get(sample)[0]
            
        if rem_dict.get(celltype) == None:
            rem_dict[celltype] = {sample: {rem: score}}
            
        elif rem_dict[celltype].get(sample) == None:
            rem_dict.get(celltype)[sample] = {rem: score}
            
        else:
            rem_dict[celltype].get(sample)[rem] = score
            

399206927it [16:22, 406420.96it/s]


In [14]:
# create template graph to fill with celltype specific values afterwards
# Graph contains gens and rems and their edges buth with no values

g = Graph()
name = g.new_vp("string")
g.vp.name = name
type = g.new_vp("string")
g.vp.type = type
start = g.new_vp("int")
g.vp.start = start
end = g.new_vp("int")
g.vp.end = end
rem = g.new_ep("string")
g.ep.rem = rem
chr = g.new_vp("string")
g.vp.chr = chr
celltypeID = g.new_ep("string")
g.edge_properties["celltypeID"] = celltypeID
celltype = g.new_ep("string")
g.edge_properties["celltype"] = celltype

done_proteins_dict = {}
done_crems = {}
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        rem_name = row[4]
        gen = row[3]
        crem = crem_dict.get(rem_name)
        if crem != None:
            crem = crem[0]
        if row[1] != "start":
            
            if crem in done_crems:
                v1 = g.vertex(done_crems.get(crem))
            elif crem != None:
                v1 = g.add_vertex()
                g.vp.name[v1] = crem
                g.vp.type[v1] = "REM"
                g.vp.chr[v1] = row[0]
                g.vp.start[v1] = crem_dict.get(rem_name)[1]
                g.vp.end[v1] = crem_dict.get(rem_name)[2]
                done_crems[crem] = int(v1)
            else:
                v1 = g.add_vertex()
                g.vp.name[v1] = rem_name
                g.vp.type[v1] = "REM"
                g.vp.chr[v1] = row[0]
                g.vp.start[v1] = int(row[1])
                g.vp.end[v1] = int(row[2])

            if done_proteins_dict.get(gen) == None:
                v2 = g.add_vertex()
                g.vp.name[v2] = gen
                g.vp.type[v2] = "Gene"
                g.vp.chr[v2] = gene_dict.get(gen)[2]
                g.vp.start[v2] = gene_dict.get(gen)[0]
                g.vp.end[v2] = gene_dict.get(gen)[1]
                done_proteins_dict[gen] = int(v2)
            else:
                v2 = g.vertex(done_proteins_dict.get(gen))

            e = g.add_edge(v1, v2)
            g.ep.rem[e] = rem_name
                
g.save(f"../data/graph-tool/template.gt")

2404862it [01:02, 38509.58it/s]


In [16]:
# load template graph and fill with celltype edge values by iterating over samples in celltype

graph = load_graph(f"../data/graph-tool/template.gt")

for celltype_id in rem_dict:
    print(celltype_id)
    
    g = Graph(graph)
    samples = rem_dict.get(celltype_id)
    celltype_name = ID_celltype_dict.get(celltype_id)
    
    for sample in samples:
        sampleID = g.new_ep("double")
        g.edge_properties[sample] = sampleID
        sample_g = g.new_vp("double")
        g.vertex_properties[sample]= sample_g
    
        for e in tqdm(g.edges()):
            rem = g.vp.name[e.source()]
            g.ep.celltypeID[e] = celltype_id
            g.ep.celltype[e] = celltype_name
            
            gene = g.vp.name[e.target()]                      # adding gene expr for specific sample
            expr = gene_expr_dict.get(gene).get(sample) 
            g.vp[sample][e.target()] = expr
            
            rem_edge_name = g.ep.rem[e]                       # adding sample name with DNase value
            g.ep[sample][e] = samples.get(sample).get(rem_edge_name)
            
    g.save(f"../data/graph-tool/{celltype_id}.gt")
#     break

CTID_0000006


2404861it [02:55, 13715.29it/s]
2404861it [02:54, 13749.76it/s]
2404861it [02:55, 13724.67it/s]


CTID_0000027


2404861it [02:56, 13637.31it/s]


CTID_0000033


...

558216it [00:40, 13799.80it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

2404861it [02:53, 13850.01it/s]
1878901it [02:16, 13821.13it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

2404861it [02:55, 13741.62it/s]
2404861it [02:53, 13886.49it/s]
523850it [00:37, 13949.39it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the co

CTID_0000050


2312510it [02:46, 13773.73it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

2404861it [02:52, 13921.65it/s]
1408708it [01:40, 13878.79it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

2404861it [02:52, 13966.15it/s]


CTID_0000054


1412889it [01:41, 13840.01it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

2321476it [02:48, 13744.03it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

2404861it [02:53, 13858.01it/s]
2199340it [02:38, 13674.07it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current v

In [ ]:
# check if all edges are the same in all samples
d = dict()
with open('../data/REMActivity_1.csv') as file:
    reader = csv.reader(file)
    counter = 0
    for row in tqdm(reader):
        if row[0] != "REMID":
            if d.get(row[1]) != None:
                d[row[1]].add(row[0])
            else:
                d[row[1]] = {row[0]}
        
check = list(d.values())[0]
for k in d.keys():
#     print(k)
    if  len(check.difference(d.get(k))) != 0:
        print(k)